In [ ]:
Assignment 1
Zohreh Ghasemi
20 January 2023

In [ ]:
# import necessary modules
import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from mizani.formatters import percent_format
from plotnine import *
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse, rmse
warnings.filterwarnings("ignore")
# turning off scientific notation

In [ ]:
df= pd.read_csv("morg2014.csv" , header = 0, sep =',')

In [ ]:
# I create some variables
df["female"] = (df["sex"] == 2)
df["w"] = df["earnwke"] / df["uhourse"]
df["agesq"] = np.power(df["age"], 2)
df["agecu"] = np.power(df["age"], 3)
df["agequ"] = np.power(df["age"], 4)

In [ ]:
# data summary
df.describe()

In [ ]:
# select occupation
df = df[df["occ2012"]==5700]

In [ ]:
df.head(5)

In [ ]:
#check missing variable in our data
df.isnull().sum()

In [ ]:
# Model 1: Linear regression
model1 = "w ~ female + age"

# Models 2-5: Multiple linear regressions

model2 = "w ~  age +female + female*age + agesq + race + prcitshp "
model3 = "w ~ female*age + agesq +female + age + race + prcitshp + ownchild + marital + grade92 + female*grade92"
model4 = "w ~ female + age + female*age + agesq + race + grade92 + female*grade92+prcitshp + ownchild + marital + agecu + agequ + female*agecu + female*agequ"


model_equations = [model1, model2, model3, model4]

In [ ]:
regs = []
for equation in model_equations:
   regs.append(smf.ols(equation, df).fit(cov_type="HC1"))

In [ ]:
stargazer.Stargazer(regs)

In [ ]:
 # RMSE of models in the full sample
w_real = df["w"].values.tolist()
RMSEs = []
for reg in regs:
    w_pred = reg.predict()
    RMSEs.append(rmse(w_real, w_pred))
RMSEs = [float('{:.3f}'.format(i)) for i in RMSEs]
RMSEs

In [ ]:
# BIC of models in the full sample
BICs = []
for reg in regs:
    BICs.append(reg.bic)
BICs = [float('{:.3f}'.format(i)) for i in BICs]
BICs

In [ ]:
results = pd.Series("model_equations")
results["BICs"] = BICs
results["RMSEs"] = RMSEs
results

In [ ]:
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold
from statsmodels.tools.eval_measures import rmse


def ols_crossvalidator(formula: str, data: pd.DataFrame, n_folds=5, average_rmse=True):


    # Get dependent variable

    y = formula.split("~")[0].strip()

    # Get statistics on the whole work data

    model = smf.ols(formula, data=data).fit()

    rsquared = model.rsquared
    # n_coefficients = model.params.shape[0]
    n_coefficients = (
        model.df_model + 1
    )  # This might differ from model.params.shape[0], because of collinear predictors
    bic = model.bic
    rmse_alldata = rmse(model.predict(), data[y])

    # Calculating test and train RMSE-s for each fold

    k = KFold(n_splits=n_folds, shuffle=False, random_state=None)

    rmse_train = []
    rmse_test = []

    for train_index, test_index in k.split(data):

        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]

        model = smf.ols(formula, data=data_train).fit()

        rmse_train.append(rmse(data_train[y], model.predict(data_train)))
        rmse_test.append(rmse(data_test[y], model.predict(data_test)))

    if average_rmse:
        rmse_train = np.mean(rmse_train)
        rmse_test = np.mean(rmse_test)

    return {
        "RMSE": rmse_alldata,
        "R-squared": rsquared,
        "BIC": bic,
        "Coefficients": n_coefficients,
        "Training RMSE": rmse_train,
        "Test RMSE": rmse_test,
    }

In [ ]:
n_fold = 4

In [ ]:
cv_list = []
for equation in model_equations:
    cv_list.append(ols_crossvalidator(equation, df, n_fold, average_rmse=False))

In [ ]:
(
    pd.DataFrame(cv_list)
    .round(3)
    .assign(
        RMSE=lambda x: x["RMSE"],
        BIC=lambda x: x["BIC"].astype(int),
        Coefficients=lambda x: x["Coefficients"].astype(int),
        Model=["Model " + str(i + 1) for i in range(len(model_equations))],
    )
    .filter(["Model", "Coefficients", "R-squared", "RMSE", "BIC"])
    .set_index("Model")
)

In [ ]:
pd.DataFrame(
    [cv["Test RMSE"] for cv in cv_list],
    index=["Model " + str(i + 1) for i in range(len(cv_list))],
    columns=["Fold" + str(i + 1) for i in range(len(cv_list[0]["Test RMSE"]))],
).assign(Average=lambda x: x.mean(axis=1)).T